In [ ]:
!pip install openai

In [2]:
import os
os.environ['OPENAI_API_KEY'] = "..."

In [ ]:
from json.decoder import JSONDecodeError
import openai
import os
import re
import json
import time

DEBUG = False

# Predefined system message
SYSTEM_MESSAGE = """

下記の前提知識とサポート記録を踏まえてサポートをしてください。
すべての会話の返答とその要約を次のJSON形式で出力してください

## 出力（JSON形式）
{{"content" : "(返答の内容)", "summary" : "(返答を要約した内容)" }}

## 会話の例
USER->機械の調子が悪いです
AI->{{"content" : "どのような機械でどんな不具合を起こしているのでしょうか？","summary" : "機械の種類と不具合の現象をユーザに確認。"}}

USER->ありがとう。(終了の暗示や挨拶や感謝の言葉なども含む)
AI->{{"content" : "どういたしまして。いつでもご相談ください。","summary" : "サポート終了。"}}

## 前提知識
  - あなたは農機具故障診断のエキスパートです。

## サポート記録
  {support_context}

"""

def create_chat_response(user_text, support_context):
  for i in range(3):  
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": SYSTEM_MESSAGE.format(support_context=support_context)
            },
            {
                "role": "user",
                "content": f"JSON形式で出力してください:[{user_text}]+"
            }
        ],
    )
    try:
      #レスポンスからJSON形式のデータだけを抜き取る
      json_str = re.search(r'\{.*\}', response.choices[0].message['content']).group(0)
      return json.loads(json_str)
    except JSONDecodeError:
      #不正なJSONデータの場合リトライする。そのとき、AIに注意を促す。
      if DEBUG: print("JSON Parse Error:",response.choices[0].message['content'])
      print("...AIの出力でエラーが発生しました。リトライします......")
      user_text += '...JSON形式のレスポンスを守ってください。'
      time.sleep(1)
      continue
    except AttributeError:
      #JSON変換できなかったもの。リストなどの場合。そのままの値をセットする。
      if DEBUG: print("Attribute Error:",response.choices[0].message['content'])
      return {
          "content": response.choices[0].message['content'],
          "summary": response.choices[0].message['content'],
          "error": "attribute error"
      }

  else:
    return []


support_context = ""
print("農機具についてお困りがあればお答えします\nなんでも聞いてください。\n")

while True:
  user_text = input("\n>")
  
  if user_text == "quit":
    break

  response = create_chat_response(user_text, support_context)
  if response:
      answer  = response['content']
      summary = response['summary']     
  else:
    print("申し訳ございません。質問の方法を変えてください。")
    continue
  
  support_context += f"- user: {user_text}\n"
  support_context += f"- AI: {summary}\n\n"
  print("\n返答:",answer)

  if DEBUG:
    print("要約:",summary)
    print("response:",response)